In [1]:
import openrouteservice
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import folium

In [2]:
class Router():
    def __init__(self):
        self.client = openrouteservice.Client(key='5b3ce3597851110001cf6248bc140663f03343e5aa7be735985a0c4f') # Specify your personal API key
    
    def GeoEncode(self, address):
        geo_coords = openrouteservice.geocode.pelias_autocomplete(self.client, address)
        return geo_coords['features'][0]['geometry']['coordinates'] # (longitude, latitude)

    def Route(self, coords):
        route = self.client.directions(coords, format = "geojson")
        return route['features'][0]['geometry']['coordinates'];

In [3]:
def getRoute(startingLocation, endingLocation, maximumDetour):
    router = Router()
    coords = [router.GeoEncode(startingLocation), router.GeoEncode(endingLocation)]
    listOfNodes = router.Route(coords)
    return listOfNodes

In [4]:
startingLocation = "601 Lakeside Rd, Fort Erie, Ontario"
endingLocation = "150 Beverley St, Toronto, Ontario"

In [5]:
res_data = pd.read_csv('resDataExample.csv')

In [6]:
def getDistance(p1,p2):
    R = 6373.0

    lat1 = radians(p1[1])
    lon1 = radians(p1[0])
    lat2 = radians(p2[1])
    lon2 = radians(p2[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

def getRestaurants(restaurants_data,route,max_distance,sort_by):
    poi_coords = restaurants_data[["lon","lat"]].values.tolist()
    distance_matrix = []
    for poi in poi_coords:
        clostest_node_in_route = np.sum(abs(np.matrix(route)-poi),axis=1).argmin()
        distance_matrix.append(getDistance(route[clostest_node_in_route],poi))
    inDistance = [dist <= max_distance for dist in distance_matrix]
    return restaurants_data[inDistance].sort_values(sort_by,ascending = False)

In [7]:
max_dist = 10
route_coords = getRoute(startingLocation,endingLocation,max_dist)

In [8]:
restaurants = getRestaurants(res_data,route_coords,max_dist,"review_score")
# restaurants = getRestaurants(res_data[res_data['restaurant_name']=="Mashawi"],route_coords,max_dist)
restaurants.head()

,restaurant_name,address,review_score,price,categories,price_range,cuisines,diets,meals,features,tags,url,lat,lon
432,Prime Steakhouse Niagara Falls,"5685 Falls Avenue, Niagara Falls, Ontario L2E ...",4.8,4.0,"['American', 'Steakhouse', 'Canadian']",[],[],[],[],[],"['baked alaska ', 'steak ', 'brussel sprouts '...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.092691,-79.071697
924,Est Restaurant,"729 Queen St E, Toronto, Ontario M4M 1H1 Canada",4.8,4.0,['Canadian'],"[10, 90]",['canadian'],[],"['dinner', 'after-hours', 'drinks']",[],"['tasting menu ', 'premium wine pairing ', 'ch...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.658935,-79.348956
928,The Tilted Dog Pub & Kitchen,"424 Parliament St, Toronto, Ontario M5A 3A2 Ca...",4.7,2.5,"['Bar', 'International', 'Pub']",[],[],[],[],[],"['brunch ', 'po boy sandwich ', 'grilled pinea...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.662918,-79.367604
136,Sushi Masayuki,"2180 Itabashi Way, Burlington, Ontario L7M 5A5...",4.7,2.5,"['Japanese', 'Seafood', 'Sushi']",[],[],[],[],[],"['sashimi ', 'spider roll ', 'california roll ...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.387090,-79.794914
926,New Orleans Seafood & Steakhouse,"267 Scarlett Rd York, Toronto, Ontario M6N 4L1...",4.7,2.5,"['Steakhouse', 'Cajun & Creole', 'Seafood']","[13, 29]","['steakhouse', 'cajun & creole', 'seafood']","['vegetarian friendly', 'gluten free options']",[],[],"['jambalaya ', 'crab cakes ', 'seafood ', ""fis...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.677764,-79.506138


In [9]:
route_reversed = [list(reversed(coord)) for coord in route_coords]
map_test = folium.Map(route_reversed[int(len(route_reversed)/2)],zoom_start=8)

folium.PolyLine(route_reversed,
                color='blue',
                weight=10,
                opacity=0.8).add_to(map_test)
for index, row in restaurants.iterrows():
    folium.Marker([row['lat'],row['lon']],color = '#d2361a', popup="Name: "+row['restaurant_name'] +" Address: "+ row['address'] ).add_to(map_test)
map_test

In [10]:
restaurants[restaurants['restaurant_name']=="Mashawi"]

,restaurant_name,address,review_score,price,categories,price_range,cuisines,diets,meals,features,tags,url,lat,lon


In [11]:
restaurants.sort_values("review_score",ascending= False)

,restaurant_name,address,review_score,price,categories,price_range,cuisines,diets,meals,features,tags,url,lat,lon
432,Prime Steakhouse Niagara Falls,"5685 Falls Avenue, Niagara Falls, Ontario L2E ...",4.8,4.0,"['American', 'Steakhouse', 'Canadian']",[],[],[],[],[],"['baked alaska ', 'steak ', 'brussel sprouts '...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.092691,-79.071697
924,Est Restaurant,"729 Queen St E, Toronto, Ontario M4M 1H1 Canada",4.8,4.0,['Canadian'],"[10, 90]",['canadian'],[],"['dinner', 'after-hours', 'drinks']",[],"['tasting menu ', 'premium wine pairing ', 'ch...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.658935,-79.348956
256,lo Presti's at Maxwell's,"Jackson st.e, Hamilton, Ontario Canada",4.7,4.0,"['Italian', 'European', 'Canadian']",[],"['italian', 'european', 'canadian']","['vegetarian friendly', 'gluten free options']","['lunch', 'dinner']",[],"['rainbow trout ', 'lamb ', 'risotto ', 'crab ...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.254303,-79.869334
434,Tide and Vine Oyster House,"3491 Portage Rd, Niagara Falls, Ontario L2J 2K...",4.7,2.5,"['Seafood', 'Wine Bar', 'Canadian']","[9, 45]","['wine bar', 'seafood', 'canadian']","['vegetarian friendly', 'gluten free options']",[],[],"['chowder ', 'oysters ', 'lobster roll ', 'map...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.121524,-79.099777
868,Beechwood Doughnuts,"165 St. Paul Street, St. Catharines, Ontario L...",4.7,2.5,"['Bakeries', 'Canadian', 'Vegetarian Friendly']",[],['canadian'],['vegetarian friendly'],[],"['takeout', 'accepts mastercard', 'accepts vis...","['doughnuts ', 'fritters ', 'cookies and cream...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.157735,-79.244674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,In Piazza Restaurant & Wine,"3530 Schmon Pky, Thorold, Ontario L2V 4Y6 Canada",3.9,2.5,['Mediterranean'],"[11, 24]",['mediterranean'],[],"['breakfast', 'lunch', 'dinner']",[],[],https://www.tripadvisor.ca/Restaurant_Review-g...,43.117886,-79.237607
936,309 Dhaba Indian Restaurant of Excellence,"309 King St W, Toronto, Ontario M5V 1J5 Canada",3.8,2.5,"['Indian', 'Seafood', 'Healthy']",[],[],[],[],[],"['butter chicken ', 'naan ', 'prawns ', 'tikka...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.646260,-79.390417
932,Spring Sushi,"10 Dundas St E, Toronto, Ontario M5B 2G9 Canada",3.7,2.5,"['Japanese', 'Seafood', 'Sushi']",[],"['japanese', 'seafood', 'sushi', 'asian', 'fus...","['vegetarian friendly', 'gluten free options']","['lunch', 'dinner', 'after-hours', 'drinks']",[],"['sushi ', 'sweet potato tempura ', 'soups ', ...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.656585,-79.380539
121,Symposium Cafe Restaurant & Lounge,"64 Hamilton St N, Waterdown, Hamilton, Ontario...",3.7,2.5,"['American', 'Bar', 'Contemporary']",[],[],[],[],[],"['rice bowl ', 'hamburger ', 'fish and chips '...",https://www.tripadvisor.ca/Restaurant_Review-g...,43.332741,-79.897259


In [40]:
res = restaurants[["restaurant_name","address","review_score","lat","lon"]].sort_values("review_score",ascending= False).reset_index().values.tolist()

In [46]:
res

[[432,
  'Prime Steakhouse Niagara Falls',
  '5685 Falls Avenue, Niagara Falls, Ontario L2E 6W7 Canada',
  4.8,
  43.09269115,
  -79.07169652],
 [924,
  'Est Restaurant',
  '729 Queen St E, Toronto, Ontario M4M 1H1 Canada',
  4.8,
  43.65893534,
  -79.3489562],
 [256,
  "lo Presti's at Maxwell's",
  'Jackson st.e, Hamilton, Ontario Canada',
  4.7,
  43.2543027,
  -79.86933370000001],
 [434,
  'Tide and Vine Oyster House',
  '3491 Portage Rd, Niagara Falls, Ontario L2J 2K5 Canada',
  4.7,
  43.1215245,
  -79.09977664],
 [868,
  'Beechwood Doughnuts',
  '165 St. Paul Street, St. Catharines, Ontario L2R 3M5 Canada',
  4.7,
  43.15773514,
  -79.24467442],
 [417,
  'LaVinia Restaurant',
  '2350 Lake Shore Blvd W, Toronto, Ontario M8V 1B6 Canada',
  4.7,
  43.6163386,
  -79.48815040000001],
 [815,
  "Terry's Cut Steakhouse",
  '300 Fourth Avenue, St. Catharines, Ontario L2R 6P9 Canada',
  4.7,
  43.154726700000005,
  -79.2976017],
 [802,
  'The Diner House 29',
  '431 Welland Ave, St. Cathar

In [44]:
for restaurant in res[:5]:
    print(restaurant[1])

Prime Steakhouse Niagara Falls
Est Restaurant
lo Presti's at Maxwell's
Tide and Vine Oyster House
Beechwood Doughnuts


In [14]:
print(res["restaurant_name"][0])
print(res["address"][0])
print(res["review_score"][0])
print(res["price"][0])
print(res["categories"][0])
print(res["tags"][0])
print(res["lat"][0])
print(res["lon"][0])

Prime Steakhouse Niagara Falls
5685 Falls Avenue, Niagara Falls, Ontario L2E 6W7 Canada
4.8
4.0
['American', 'Steakhouse', 'Canadian']
['baked alaska ', 'steak ', 'brussel sprouts ', 'loaded baked potato ', 'chocolate cake ', 'scallops ', 'rice wine ', 'rainbow roll ', 'fries ', 'caesar salad ', 'server james ', 'the crowne plaza ', 'chicago style ', 'anniversary dinner ', 'complimentary dessert ', 'cooked to perfection ', 'food was amazing ']
43.09269115
-79.07169652


In [16]:
res_name_dict = res["restaurant_name"]
res_address_dict = res["address"]
res_review_dict = res["review_score"]
res_lon_dict = res["lon"]
res_lat_dict = res["lat"]
string = '{ "restaurantNames":"' + str(res_name_dict) + '", "restaurantAddresses":"' + str(res_address_dict) + '","restaurantReviews":"' + str(res_review_dict) + '","restaurantLons":"' + str(res_lon_dict) + '","restaurantLats":"' + str(res_lat_dict) + '"}'

In [17]:
string

'{ "restaurantNames":"{0: \'Prime Steakhouse Niagara Falls\', 1: \'Est Restaurant\', 2: "lo Presti\'s at Maxwell\'s", 3: \'Tide and Vine Oyster House\', 4: \'Beechwood Doughnuts\', 5: \'LaVinia Restaurant\', 6: "Terry\'s Cut Steakhouse", 7: \'The Diner House 29\', 8: \'Pho 18\', 9: \'Buried Treasure Cafe\', 10: \'ALO RESTAURANT\', 11: \'New Orleans Seafood & Steakhouse\', 12: \'Sushi Masayuki\', 13: \'The Tilted Dog Pub & Kitchen\', 14: \'AG Inspired Cuisine\', 15: \'Scoops Restaurant\', 16: \'Rise Above\', 17: \'Stelvio\', 18: \'The Karma Kameleon Gastropub\', 19: "Turtle Jack\'s Niagara Falls", 20: \'Richmond Station\', 21: \'Madrina Bar Y Tapas\', 22: \'Jacques Bistro Du Parc\', 23: \'Olive Us Greek Restaurant\', 24: \'Downtown Bistro & Grill\', 25: \'Mythos Greek Cuisine Winebar\', 26: "Kelly\'s Bake Shoppe", 27: \'Wellington Court Restaurant + Catering\', 28: \'Amaya Express\', 29: \'Berkeley North Restaurant\', 30: \'The Blue Line\', 31: \'Luci Restaurant\', 32: \'Piatto Bistro\'